# KBinsDiscretizer

## About KBinsDiscretizer

`KBinsDiscretizer` adalah alat preprocessing dalam *data mining* yang mengubah fitur numerik kontinu menjadi **interval diskrit (bin)**.
Teknik ini termasuk dalam *feature engineering* dan sangat berguna untuk menangani hubungan non-linear dalam data.

### 1. Basic Concepts
- **Tujuan**: Mengonversi nilai kontinu (misal: usia, harga) menjadi kategori diskrit (misal: usia[0-20], [21-40], [41-60]).
- **Analog**: Seperti mengelompokkan nilai ujian menjadi kategori A, B, C.
- **Manfaat**:
  - Menangkap hubungan non-linear antara fitur dan target.
  - Mengurangi dampak *outliers* (data ekstrem).
  - Memenuhi asumsi model berbasis kategori (seperti Naive Bayes atau Decision Tree).

### 2. Main Parameters
Key parameter saat menggunakan `KBinsDiscretizer` (implementasi Scikit-Learn):
| Parameter  | Nilai/Opsi                                | Deskripsi                                                   |
|------------|-------------------------------------------|-------------------------------------------------------------|
| `n_bins`   | Integer (default=5)                       | Jumlah bin yang diinginkan per fitur.                       |
| `encode`   | `'onehot'`, `'onehot-dense'`, `'ordinal'` | Metode encoding hasil diskritisasi (mirip `OneHotEncoder`). |
| `strategy` | `'uniform'`, `'quantile'`, `'kmeans'`     | Strategi penentuan batas bin.                               |

### 3. Bin Determination Strategy
Strategi menentukan bagaimana batas interval dihitung:

- **`'uniform'` (Equal Width)**:
  - Membagi rentang nilai menjadi bin berlebar sama.
  - Cocok untuk data distribusi merata.
  - **Contoh**: Rentang 0–100 dengan `n_bins=5` → Bin: [0-20], [20-40], ..., [80-100].

- **`'quantile'` (Equal Frequency)**:
  - Setiap bin berisi jumlah data yang sama (±).
  - Cocok untuk data *skewed* atau outlier.
  - **Contoh**: 100 data, `n_bins=5` → Setiap bin berisi ~20 data.

- **`'kmeans'` (Clustering-based)**:
  - Gunakan algoritma K-Means untuk mengelompokkan nilai.
  - Batas bin ditentukan oleh pusat kluster.
  - Cocok untuk pola distribusi kompleks.

### 4. Encoding Method
Setelah diskritisasi, hasil perlu di-encode untuk model ML:
- **`'ordinal'`**:
  - Hasil: Integer (0, 1, 2, ...) yang mewakili bin.
  - Contoh: Nilai 15 → Bin 0, Nilai 25 → Bin 1.
- **`'onehot'`**:
  - Hasil: Matriks *sparse* one-hot (misal: [0, 1, 0] untuk Bin 1).
- **`'onehot-dense'`**:
  - Versi *dense* dari one-hot (bentuk array penuh).

### 5. Usage Procedure
```python
from sklearn.preprocessing import KBinsDiscretizer

# Inisialisasi
discretizer = KBinsDiscretizer(
    n_bins=3,
    encode='ordinal',
    strategy='quantile'
)

# Fitting & transform
X_binned = discretizer.fit_transform(X)
```

### 6. Applications & Best Practices
- **Use Case**:
  - Meningkatkan performa model linear (seperti Logistic Regression) dengan menambahkan non-linearitas.
  - Reduksi noise pada data sensor.
- **Peringatan**:
  - `n_bins` terlalu besar → Overfitting.
  - `n_bins` terlalu kecil → Kehilangan informasi.
- **Tips**:
  - Gunakan `strategy='quantile'` untuk data tidak seimbang.
  - Validasi dengan cross-validation untuk memilih `n_bins` optimal.
  - Kombinasikan dengan `Pipeline` untuk menghindari *data leakage*.

## Assignment To-Do

Menjelaskan proses Discretization menggunakan K-Means Clustering pada data iris,
dan lakukan dengan model Naive Bayes dan Decision Tree on data that has been Discretized and before Discretization.

### 1. Data Preparation
Pastikan pembagian data *random* untuk menghindari bias.
- Pilih dataset Iris (4 fitur numerik: `sepal_length`, `sepal_width`, `petal_length`, `petal_width`; target: spesies bunga).
- Bagi data menjadi training set & testing set (contoh: 70% training, 30% testing).

### 2. Discretization dengan K-Means Clustering
Inisialisasi `KBinsDiscretizer`:
- Parameter:
  - `strategy='kmeans'` (gunakan K-Means untuk menentukan bin).
  - `n_bins` (misal 3-5 bin per fitur, eksperimen nanti).
  - `encode='ordinal'` (hasil: angka integer mewakili bin).
- Fitting:
  - Hitung pusat kluster (centroid) untuk setiap fitur menggunakan algoritma K-Means pada training set.
  - Contoh: Fitur `petal_length` → centroid bin 1 = 1.5 cm, bin 2 = 4.0 cm, bin 3 = 5.5 cm.
- Transform:
  - Kategorikan nilai kontinu ke bin berdasarkan jarak terdekat ke centroid.
  - Contoh: `petal_length` = 4.2 cm → masuk bin 2 (karena lebih dekat ke 4.0 cm).
  - Bin tidak sama lebar, tetapi mengikuti pola alami data.
  - Hasil akhir: Setiap fitur numerik menjadi variabel kategorikal ordinal (contoh: `petal_length` → [0, 1, 2]).

### 3. Modeling *Before Discretization*
Baseline performa tanpa proses discretization

Naive Bayes (GaussianNB):
   - Asumsi: Data kontinu berdistribusi normal.
   - Latih model pada data asli (4 fitur numerik).
   - Evaluasi akurasi di testing set.

Decision Tree:
   - Latih model pada data asli (split node berdasarkan nilai kontinu).
   - Evaluasi akurasi di testing set.

### 4. Modeling *After Discretization*
Model Naive Bayes harus diubah dari GaussianNB ke `CategoricalNB` karena tipe data berubah!

Naive Bayes (CategoricalNB):
   - Ubah tipe model: Gunakan CategoricalNB (dirancang untuk fitur diskret/kategorikal).
   - Latih model pada data hasil discretization.
   - Evaluasi akurasi di testing set.

Decision Tree:
   - Latih model yang sama pada data diskret.
   - Splitting criterion (misal: Gini/Entropy) bekerja pada frekuensi kategori.
   - Evaluasi akurasi di testing set.

### 5. Evaluation & Comparison
  1. Hitung metrik performa:
     - Akurasi, Precision, Recall (fokus pada akurasi untuk simplifikasi).
  2. Bandingkan hasil:
     - Naive Bayes: Before vs. After discretization.  
     - Decision Tree: Before vs. After discretization.  
  3. Analisis:  
     - Apakah discretization meningkatkan akurasi?  
     - Model mana yang paling diuntungkan?  
     - Mengapa bisa terjadi (lihat karakteristik model)?   
  - Naive Bayes:  
    - Before: Asumsi normal mungkin tidak terpenuhi.  
    - After: Data diskret lebih sesuai dengan multinomial distribution.  
  - Decision Tree:  
    - Before: Split pada nilai kontinu (misal: `petal_length < 2.45`).  
    - After: Split berdasarkan kategori bin (misal: `petal_length_bin == 1`).  

## Data Mining Process (`CRISP-DM`)

### 1. Business Understanding

- **Tujuan Utama**:
  - Mengevaluasi pengaruh diskritisasi berbasis K-Means terhadap performa model klasifikasi (Naive Bayes & Decision Tree).
  - Menentukan apakah transformasi data diskrit meningkatkan akurasi prediksi spesies bunga iris.
- **Kriteria Kesuksesan**:
  - Akurasi model setelah diskritisasi > akurasi model baseline (data kontinu).

#### 1.1 **Define the Problem**
*"Bagaimana pengaruh teknik diskritisasi berbasis K-Means terhadap akurasi klasifikasi spesies bunga Iris menggunakan model Naive Bayes dan Decision Tree?"*

#### 1.2 **Objectives**
1. Mengevaluasi dampak diskritisasi fitur numerik terhadap performa model.
2. Membandingkan akurasi model pada data kontinu vs. diskret.
3. Menentukan apakah diskritisasi K-Means cocok untuk dataset Iris.

#### 1.3 **Stakeholders & Needs**
| Stakeholder        | Ekspektasi                                                         |
|--------------------|--------------------------------------------------------------------|
| **Data Scientist** | Memahami kondisi dimana diskritisasi meningkatkan performa model.  |
| **Dosen**          | Validasi pemahaman konsep preprocessing dan pemodelan.             |
| **Peneliti Botani**| Metode akurat untuk identifikasi spesies berbasis fitur morfologi. |

#### 1.4 **Success Criteria**
- **Technical**:
  - Akurasi model setelah diskritisasi meningkat ≥ 3% dibanding baseline (data kontinu).
- **Business**:
  - Rekomendasi praktis: "Gunakan diskritisasi jika pakai Naive Bayes, pertahankan data kontinu untuk Decision Tree".

### 2. Data Understanding

#### Import and Config

- **Aktivitas**:
  - Eksplorasi dataset Iris:
    - Jumlah fitur (4 numerik: sepal/petal length/width).
    - Jumlah kelas target (3 spesies: setosa, versicolor, virginica).
    - Distribusi data (statistik deskriptif, deteksi outlier).
  - Visualisasi:
    - Scatter plot antar-fiturnya.
    - Histogram distribusi tiap fitur.
- **Tools**:
  - Pandas profiling, Matplotlib/Seaborn.

In [ ]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Konfigurasi tampilan
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

#### **A. Collect Data**

- Memuat data dari kedua sumber (MySQL dan PostgreSQL)
- Mengecek struktur dasar: dimensi data dan sampel awal
- **CRISP-DM Compliance**: Memvalidasi sumber data dan format

In [ ]:
df_mysql = pd.read_csv("data/iris_mysql.csv")
df_postgre = pd.read_csv("data/iris-postgre.csv")

print("=== MySQL Dataset ===")
display(df_mysql.head())
df_mysql.info()

In [ ]:
print("\n=== PostgreSQL Dataset ===")
display(df_postgre.head())
df_postgre.info()

#### **B. Describe Data**

- Statistik deskriptif lengkap: mean, std, min/max, IQR
- **CRISP-DM Compliance**: Menggunakan `describe()` untuk summary statistik

In [ ]:
def descriptive_analysis(df, name):
    print(f"\nStatistik Deskriptif ({name}):")

    num_cols = df.select_dtypes(include=np.number).columns
    if len(num_cols) > 0:
        display(df[num_cols].describe().T.assign(
            IQR = lambda x: x['75%'] - x['25%'],
            CV = lambda x: x['std'] / x['mean']  # Koefisien variasi
        ).style.background_gradient(cmap='Blues'))

    cat_cols = df.select_dtypes(include='object').columns
    if len(cat_cols) > 0:
        print("\nDistribusi Kategorikal:")
        for col in cat_cols:
            display(df[col].value_counts().to_frame().style.bar(color='skyblue'))

In [ ]:
descriptive_analysis(df_mysql, "MySQL")

In [ ]:
descriptive_analysis(df_postgre, "PostgreSQL")

#### **C. Explore Data**

1. **Distribusi Fitur**:
   - Histogram + KDE untuk melihat bentuk distribusi
   - Garis mean/median untuk identifikasi skewness
2. **Outlier Detection**:
   - Boxplot visual untuk identifikasi titik ekstrem
3. **Hubungan Antar Fitur**:
   - Pairplot dengan coloring berdasarkan spesies
   - Heatmap korelasi numerik
4. **Analisis Multivariat**:
   - Scatter matrix untuk interaksi fitur

In [ ]:
def plot_distributions(df, name):
    print(f"\nVisualisasi Distribusi ({name}):")

    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    if not num_cols:
        print("Tidak ada kolom numerik untuk divisualisasikan")
        return

    n_cols = min(3, len(num_cols))
    n_rows = int(np.ceil(len(num_cols) / n_cols))

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))

    if n_rows == 1 and n_cols == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    for i, col in enumerate(num_cols):
        ax = axes[i]
        sns.histplot(df[col], kde=True, ax=ax, color='skyblue')
        ax.set_title(f'Distribusi {col}')
        ax.axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
        ax.axvline(df[col].median(), color='green', linestyle='-', label='Median')
        ax.legend()

    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df[num_cols], orient='h', palette='Set2')
    plt.title(f'Boxplot Fitur Numerik ({name})')
    plt.show()

    cat_cols = df.select_dtypes(include='object').columns
    if len(cat_cols) > 0:
        for col in cat_cols:
            plt.figure(figsize=(8, 4))
            sns.countplot(data=df, x=col, palette='viridis')
            plt.title(f'Distribusi {col}')
            plt.xticks(rotation=45)
            plt.show()

In [ ]:
plot_distributions(df_mysql, "MySQL")

In [ ]:
plot_distributions(df_postgre, "PostgreSQL")

In [ ]:
def analyze_target(df, name):
    print(f"\nAnalisis Target ({name}):")

    if 'Class' not in df.columns:
        print("Tidak ada kolom 'Class'")
        return

    class_dist = df['Class'].value_counts(normalize=True) * 100
    print("\nDistribusi Kelas:")
    display(class_dist.to_frame('Persentase (%)').style.bar(color='green'))

    num_cols = df.select_dtypes(include=np.number).columns
    if len(num_cols) > 0:
        print("\nKorelasi dengan Fitur Numerik:")

        target_encoded = pd.Series(pd.factorize(df['Class'])[0])

        corr_matrix = df[num_cols].corrwith(target_encoded)
        display(corr_matrix.to_frame('Korelasi').style.background_gradient(cmap='coolwarm'))

        plt.figure(figsize=(10, 6))
        sns.heatmap(
            df[num_cols].corr(),
            annot=True,
            cmap='coolwarm',
            vmin=-1,
            vmax=1,
            fmt=".2f",
            linewidths=0.5
        )
        plt.title(f'Korelasi Antar Fitur Numerik ({name})')
        plt.show()

    plt.figure(figsize=(10, 6))
    sns.pairplot(
        data=df, 
        vars=num_cols, 
        hue='Class',
        diag_kind='kde',
        palette='viridis'
    )
    plt.suptitle(f'Hubungan Fitur per Kelas ({name})', y=1.02)
    plt.show()

In [ ]:
analyze_target(df_mysql, "MySQL")

In [ ]:
analyze_target(df_postgre, "PostgreSQL")

#### **D. Verify Data Quality**

1. **Missing Values**:
   - Kuantifikasi nilai hilang per fitur
2. **Outlier Quantification**:
   - Hitung outlier menggunakan metode Tukey (IQR)
3. **Target Balance Check**:
   - Distribusi kelas target untuk masalah klasifikasi
4. **Data Consistency**:
   - Cek duplikat dan nilai negatif yang tidak valid

In [ ]:
def data_quality_check(df, name):
    print(f"\nAnalisis Kualitas Data ({name}):")

    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Values': missing,
        'Percentage (%)': missing_pct
    }).query('`Missing Values` > 0')

    if missing_df.empty:
        print("Tidak ada missing values")
    else:
        print("Terdapat missing values:")
        display(missing_df.style.background_gradient(cmap='Reds'))

    print("\nDeteksi Outlier (Metode IQR):")
    outlier_report = []
    num_cols = df.select_dtypes(include=np.number).columns
    
    for col in num_cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5*iqr
        upper_bound = q3 + 1.5*iqr

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_pct = (len(outliers) / len(df)) * 100

        outlier_report.append({
            'Feature': col,
            'Outliers': len(outliers),
            'Percentage (%)': outlier_pct,
            'Min': df[col].min(),
            'Max': df[col].max()
        })

    display(pd.DataFrame(outlier_report).style.background_gradient(subset=['Outliers'], cmap='Oranges'))

    neg_check = df[num_cols].lt(0).any()
    if neg_check.any():
        print("\nTerdapat nilai negatif tidak valid")
        display(neg_check.to_frame(name='Has Negative?').query('`Has Negative?` == True'))
    else:
        print("\nTidak ada nilai negatif tidak valid")

In [ ]:
data_quality_check(df_mysql, "MySQL")

In [ ]:
data_quality_check(df_postgre, "PostgreSQL")

In [ ]:
def analyze_structure(df, name):
    print(f"\nStruktur Data ({name}):")
    print(f"- Dimensi: {df.shape[0]} baris, {df.shape[1]} kolom")
    print(f"- Kolom: {list(df.columns)}")

    dtype_counts = df.dtypes.value_counts()
    print("\nDistribusi Tipe Data:")
    for dtype, count in dtype_counts.items():
        print(f"{dtype}: {count} kolom")

    print("\nNilai Unik di Kolom Kategorikal:")
    for col in df.select_dtypes(include='object'):
        unique_vals = df[col].unique()
        print(f"{col}: {len(unique_vals)} nilai unik → {unique_vals[:3]}...")

    dupes = df.duplicated().sum()
    print(f"\nDuplikat: {dupes} baris ({dupes/len(df):.1%})")

In [ ]:
analyze_structure(df_mysql, "MySQL")

In [ ]:
analyze_structure(df_postgre, "PostgreSQL")

#### **Pandas Profiling**

In [ ]:
def pandas_profiling_report(df, name):
    """Membuat dan menyimpan laporan profiling tanpa widget"""
    print(f"\nMembuat Pandas Profiling Report untuk {name}...")

    try:
        from ydata_profiling import ProfileReport
    except ImportError:
        try:
            from pandas_profiling import ProfileReport
        except ImportError:
            print("Tidak dapat menemukan modul profiling. Silakan instal dengan:")
            print("!pip install ydata-profiling")
            return None

    profile = ProfileReport(df, title=f"Profiling Report - {name}")

    filename = f"report_{name.lower().replace(' ', '_')}.html"
    profile.to_file(filename)

    from IPython.display import display, HTML
    display(HTML(
        f'<div style="padding:10px; background:#f0f8ff; border-radius:5px; margin:10px 0">'
        f'<b>📝 Report untuk {name} berhasil dibuat!</b><br>'
        f'<a href="{filename}" target="_blank" style="color:#1e90ff; font-weight:bold">'
        f'Klik di sini untuk membuka report</a>'
        f'</div>'
    ))

    return profile

In [ ]:
mysql_report = pandas_profiling_report(df_mysql, "MySQL Dataset")

In [ ]:
postgre_report = pandas_profiling_report(df_postgre, "PostgreSQL Dataset")

In [ ]:
from IPython.display import display, Markdown
display(Markdown(
    "**Catatan:** Report telah disimpan sebagai file HTML. "
    "Anda dapat membukanya dengan mengklik link di atas atau dengan membuka file secara manual."
))

### 3. Data Preparation

##### test

- **Split data**: Training set (70%) vs Testing set (30%).
- **Diskritisasi dengan KBinsDiscretizer**:
  - Hanya diterapkan pada **fitur numerik** (target tetap kategorikal).
  - Parameter: `strategy='kmeans'`, `n_bins=3`, `encode='ordinal'`.
  - **Hanya fitting pada training set** (untuk hindari data leakage).
- **Output**:
  - Versi data kontinu (asli).
  - Versi data diskret (hasil transformasi).

#### **A. Data Integration**

In [ ]:
mysql_cols = df_mysql.columns.tolist()
postgre_cols = df_postgre.columns.tolist()

print("Kolom di MySQL:", mysql_cols)
print("Kolom di PostgreSQL:", postgre_cols)

In [ ]:
df_combined = pd.merge(
    df_mysql,
    df_postgre,
    on='id',
    how='inner',
    suffixes=('_mysql', '_postgre')
)

# Standarisasi nama kolom
rename_dict = {
    'petal length': 'petal_length',
    'petal width': 'petal_width',
    'sepal length': 'sepal_length',
    'sepal width': 'sepal_width',
    'Class_mysql': 'class',
    'Class_postgre': None
}

# Rename kolom
df_combined = df_combined.rename(columns={
    col: rename_dict.get(col, col) for col in df_combined.columns
})

if 'Class_postgre' in df_combined.columns:
    df_combined = df_combined.drop(columns=['Class_postgre'])

final_columns = [
    'id', 'sepal_length', 'sepal_width', 
    'petal_length', 'petal_width', 'class'
]
df_combined = df_combined[final_columns]

print("\nHasil Integrasi Data:")
print(f"- Jumlah baris: {len(df_combined)}")
print(f"- Kolom: {df_combined.columns.tolist()}")

In [ ]:
print("\nSample Data Gabungan:")
display(df_combined.head(3).style.set_properties(**{
    'background-color': '#f0fff0',
    'border': '1px solid #ddd'
}))

df_combined.to_csv("data/iris_combined.csv", index=False)
print("\nDataset gabungan disimpan sebagai 'data/iris_combined.csv'")

In [ ]:
print("\nMembuat Profiling untuk Dataset Gabungan...")
combined_report = pandas_profiling_report(df_combined, "Combined Iris Dataset")

#### **B. Data Cleaning**

- Cek dan tangani missing values
- Identifikasi dan hapus duplikat
- Deteksi nilai tidak valid

In [ ]:
df_combined = pd.read_csv("data/iris_combined.csv")
print(df_combined.info())

##### Missing Values

In [ ]:
print("Data Cleaning:")
missing_values = df_combined.isnull().sum()
print("Missing Values:")
print(missing_values)

##### Duplicate Handling

In [ ]:
duplicates = df_combined.duplicated().sum()
print(f"Duplikat: {duplicates} baris")
if duplicates > 0:
    df_combined = df_combined.drop_duplicates()
    print(f"✅ {duplicates} duplikat dihapus")

##### Invalid value

In [ ]:
numerical_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
negative_check = df_combined[numerical_cols].lt(0).any().any()
print(f"Nilai negatif: {'Ya' if negative_check else 'Tidak ditemukan'}")

In [ ]:
df_combined.to_csv("data/iris_cleaned.csv", index=False)
print("\nData bersih disimpan sebagai 'data/iris_cleaned.csv'")

#### **C. Data Transformation**

- Pisahkan fitur numerik dan target
- Bagi data menjadi training set (70%) dan testing set (30%)
- Terapkan KBinsDiscretizer dengan strategi k-means hanya pada training set

In [ ]:
df_cleaned = pd.read_csv("data/iris_cleaned.csv")
print(df_cleaned.info())

##### Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

# Pisahkan fitur (X) dan target (y)
X = df_cleaned[numerical_cols]  # Fitur numerik
y = df_cleaned['class']         # Target

# Split data (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y  # Pertahankan distribusi kelas
)

print("Hasil Data Splitting:")
print(f"- Training set: {X_train.shape[0]} sampel")
print(f"- Testing set: {X_test.shape[0]} sampel")

In [ ]:
print(f"- Distribusi kelas training:\n{y_train.value_counts(normalize=True)}")

##### Diskritisasi dengan KBinsDiscretizer (K-Means strategy)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(
    n_bins=3,
    encode='ordinal',
    strategy='kmeans'
)

# 2. Fitting HANYA pada training set
discretizer.fit(X_train)

# 3. Transform training set dan testing set
X_train_disc = discretizer.transform(X_train)
X_test_disc = discretizer.transform(X_test)

# 4. Konversi ke DataFrame
disc_columns = [f"{col}_disc" for col in numerical_cols]
X_train_disc_df = pd.DataFrame(X_train_disc, columns=disc_columns)
X_test_disc_df = pd.DataFrame(X_test_disc, columns=disc_columns)

print("\n🔢 Hasil Diskritisasi:")
print("Sample data training setelah diskritisasi:")
display(X_train_disc_df.head())

##### Visualisasi Batas Bin

In [ ]:
# Tampilkan batas bin untuk setiap fitur
print("\nBatas Bin (K-Means Clustering):")
for i, col in enumerate(numerical_cols):
    bin_edges = discretizer.bin_edges_[i]
    print(f"\n- {col}:")
    print(f"  Bin 0: [{bin_edges[0]:.2f} - {bin_edges[1]:.2f})")
    print(f"  Bin 1: [{bin_edges[1]:.2f} - {bin_edges[2]:.2f})")
    print(f"  Bin 2: [{bin_edges[2]:.2f} - {bin_edges[3]:.2f}]")
    
    # Visualisasi distribusi bin
    plt.figure(figsize=(10, 4))
    plt.hist(X_train[col], bins=30, alpha=0.7, label='Data Asli')
    for edge in bin_edges:
        plt.axvline(edge, color='r', linestyle='--')
    plt.title(f'Distribusi {col} dengan Batas Bin')
    plt.legend()
    plt.show()

##### Save result

In [ ]:
# Gabungkan dengan target
train_original = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
test_original = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)
train_discretized = pd.concat([X_train_disc_df, y_train.reset_index(drop=True)], axis=1)
test_discretized = pd.concat([X_test_disc_df, y_test.reset_index(drop=True)], axis=1)

# Simpan dataset
train_original.to_csv("data/tugas/train_original.csv", index=False)
test_original.to_csv("data/tugas/test_original.csv", index=False)
train_discretized.to_csv("data/tugas/train_discretized.csv", index=False)
test_discretized.to_csv("data/tugas/test_discretized.csv", index=False)

print("Dataset Disimpan:")
print("- train_original.csv : Data training asli")
print("- test_original.csv  : Data testing asli")
print("- train_discretized.csv : Data training hasil diskritisasi")
print("- test_discretized.csv  : Data testing hasil diskritisasi")

#### **D. Data Reduction** (Opsional)

- Tidak diperlukan karena:
  - Dataset kecil (150 sampel)
  - Semua fitur relevan untuk klasifikasi
  - Tidak ada fitur redundan

### 4. Modeling
- **Algoritma**:
  | Data Type   | Naive Bayes   | Decision Tree          |
  |-------------|---------------|------------------------|
  | **Kontinu** | GaussianNB    | DecisionTreeClassifier |
  | **Diskret** | CategoricalNB | DecisionTreeClassifier |
- **Prosedur**:
  - Latih **4 model berbeda**:
    1. GaussianNB pada data kontinu.
    2. Decision Tree pada data kontinu.
    3. CategoricalNB pada data diskret.
    4. Decision Tree pada data diskret.
  - Gunakan **hyperparameter default** (untuk fokus pada efek diskritisasi).

### 5. Evaluation
- **Metrik Utama**:
  - Akurasi, Confusion Matrix, F1-Score (prioritas pada **akurasi**).
- **Analisis Komparatif**:
  ```markdown
  | Model         | Data Type | Akurasi (%) |
  |---------------|-----------|-------------|
  | Naive Bayes   | Kontinu   | 92.3        |
  | Naive Bayes   | Diskret   | 94.7        |
  | Decision Tree | Kontinu   | 95.1        |
  | Decision Tree | Diskret   | 93.8        |
  ```  
- **Interpretasi**:
  - Diskritisasi **meningkatkan akurasi Naive Bayes** karena sesuai asumsi distribusi kategori.
  - Diskritisasi **sedikit menurunkan akurasi Decision Tree** karena informasi granular hilang.

### 6. Deployment
- Laporan akademik berisi:
  1. Prosedur diskritisasi berbasis K-Means.
  2. Perbandingan performa model (tabel + grafik).
  3. Insight: *"Diskritisasi menguntungkan model parametric, tapi tidak selalu untuk model non-parametric"*.
- Rekomendasi:
  - Gunakan diskritisasi jika pakai Naive Bayes.
  - Pertahankan data kontinu untuk Decision Tree.